In [1]:
import getpass, os

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"Enter {var}: ")

_set_env("OPENAI_API_KEY")

In [15]:
from typing import Annotated

from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

class State(TypedDict):
    messages: Annotated[list, add_messages]

graph_builder = StateGraph(State)

In [16]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

def chatbot(state: State) -> dict:
    return {"messages": llm.invoke(state["messages"])}

graph_builder.add_node("chatbot", chatbot)

In [ ]:
graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", END)

graph = graph_builder.compile()

In [ ]:
from IPython.display import Image, display

display(Image(graph.get_graph().draw_mermaid_png()))

In [32]:
def stream_graph_updates(user_input: str):
    for event in graph.stream({"messages": [{"role": "user", "content": user_input }]}):
        for value in event.values():
            print("Assistant: ", value["messages"].content)

stream_graph_updates("안녕 오늘은 무슨 요일이야?")


Assistant:  안녕하세요! 오늘은 2023년 10월 4일 수요일입니다. 도움이 필요하시면 언제든지 말씀해 주세요!


In [34]:
while True:
    user_input = input("You: ")
    print("User: ", user_input)
    if user_input.lower() in ["quit", "exit", "q"]:
        print("goodbye!")
        break
    stream_graph_updates(user_input)

User:  랭체인이 뭐야
Assistant:  랭체인(Chain of Thought, CoT)은 주로 인공지능과 자연어 처리 분야에서 사용되는 개념으로, 문제 해결이나 질문에 대한 답변을 도출하는 과정에서의 사고 과정을 나타냅니다. 이 접근법은 모델이 단순히 정답을 제시하는 것이 아니라, 그 과정에서의 논리적 추론이나 단계별 사고를 보여줌으로써 더 정확하고 신뢰할 수 있는 결과를 도출할 수 있도록 돕습니다.

랭체인은 특히 복잡한 문제를 해결할 때 유용하며, 모델이 중간 단계를 명시적으로 표현함으로써 사용자가 결과를 이해하고 검증할 수 있는 기회를 제공합니다. 이러한 방식은 AI 모델의 투명성을 높이고, 결과의 신뢰성을 향상시키는 데 기여할 수 있습니다. 

랭체인은 다양한 AI 응용 프로그램에서 활용될 수 있으며, 특히 대화형 AI, 문제 해결 시스템, 교육용 도구 등에서 그 효과를 발휘합니다.
User:  bia의 원리에 대해서 알려줄래
Assistant:  BIA(Bioelectrical Impedance Analysis)는 생체 전기 임피던스 분석의 약자로, 신체의 조성을 평가하는 방법 중 하나입니다. 이 원리는 신체에 미세한 전류를 흘려보내고, 그 전류가 신체를 통과할 때의 저항(임피던스)을 측정하는 것입니다. BIA의 기본 원리는 다음과 같습니다:

1. **전기 전도성**: 인체는 수분, 지방, 근육 등 다양한 성분으로 이루어져 있으며, 이들 성분은 전기 전도성이 다릅니다. 예를 들어, 근육은 수분 함량이 높아 전기가 잘 통과하지만, 지방은 전기가 잘 통하지 않습니다.

2. **전류의 흐름**: BIA 장비는 신체에 저전압의 전류를 흘려보내고, 이 전류가 신체를 통과할 때의 저항을 측정합니다. 이 저항 값은 신체의 수분 함량, 지방량, 근육량 등을 추정하는 데 사용됩니다.

3. **데이터 해석**: 측정된 임피던스 값은 알고리즘을 통해 해석되어, 체지방률, 근육량, 체수분량 등의 정보를 제공합니다. 이를 통해 개인의 신체 조성을 평가하고, 건